# Examen Final

- Prender cámaras.
- Puede utilizar sus apuntes.
- Puede utilizar los notebooks del curso.
- La alumna o alumno que sea sorprendido solicitando ayuda a cualquier persona, sea ella compañera o no, será calificada con un 2.
- Cada pregunta vale un punto.
- La nota mínima es un 2.
- La nota se determinará con la siguiente fórmula $nota = \min\left(puntos + 2, 7\right)$
- Todas las funciones necesarias para resolver los ejercicios están ya importadas al notebook.

## Librerías

In [21]:
from finrisk import examen_final as ef
from scipy.interpolate import interp1d
import modules.hull_white as hw
import pandas as pd
import numpy as np
import textwrap
import random
import math

## Curva Cero Cupón

Los plazos están en días y las tasas en convención exp act/365.

In [46]:
df_curva = pd.read_excel('data/20201012_built_sofr_zero.xlsx')

In [47]:
df_curva.head()

,plazo,tasa,df
0,1,0.000811,0.999998
1,7,0.000841,0.999984
2,14,0.000780,0.999970
3,21,0.000774,0.999955
4,33,0.000781,0.999929


In [48]:
curva = interp1d(df_curva['plazo'], df_curva['tasa'], 'linear', fill_value='extrapolate')

## Cálculo Opción

Para identificar si es una Call o una Put se usa un `enum`.

In [49]:
c_p = hw.CallPut.CALL
c_p = hw.CallPut.PUT

Ver toda la documentación:

In [50]:
print(hw.zcb_call_put.__doc__)


    Calcula el valor de una call o una put sobre un bono cero cupón en el modelo de HW.
    
    params:
    
    - c_p: indica si es la opción es Call o Put. Es un `enum` de tipo CallPut. Ejemplo, c_p = CallPut.CALL.
    - strike: es el strike de la opción. Se ingresa como número. Un strike del 90% se ingresa como .9.
    - r0: es la tasa corta al momento de valorizar la opción (t = 0).
    - to: instante de tiempo en que vence la opción, expresado en años.
    - tb: instante de tiempo en que vence el bono subyacente, expresado en años. Debe ser tb > to.
    - zo: factor de descuento de mercado a tiempo t = 0 hasta to.
    - zb: factor de descuento de mercado a tiempo t = 0 hasta tb.
    - gamma: parámetro gamma del modelo HW.
    - sigma: parámetro sigma del modelo HW.
    
    return:
    
    -  el valor de la opción.
    


## Valores `gamma` y `sigma`

In [51]:
gamma = 1
sigma = .005

In [52]:
strikes = [.98, .981, .982, .983, .984, .985, .99, .991, .992, .993]
for k in strikes:
    print(k, hw.zcb_call_put(
        hw.CallPut.CALL,
        k,
        curva(.0001),
        1,
        2,
        math.exp(-curva(365)),
        math.exp(-curva(730) * 730 / 365.0),
        gamma,
        sigma
    ))

0.98 0.019553575799963863
0.981 0.0185542779311344
0.982 0.017554980062305048
0.983 0.016555682193475696
0.984 0.015556384324646233
0.985 0.014557086455816881
0.99 0.0095605971116699
0.991 0.008561299242840548
0.992 0.007562001374011196
0.993 0.006562703505181733


## Preguntas

In [29]:
try:
    q = ef.get_questions()
except Exception as e:
    print(str(e))

Sólo puedes pedir las preguntas 1 vez.


In [30]:
for qq in q:
    print(f'{qq[0]}\n')
    print(textwrap.fill(f'{qq[1]}', 80))
    print('\n')

Pregunta 1:

Usando la curva cero cupón entregada y parámetros gamma = 1.0 y sigma = 0.5%
valorice una call a 1Y sobre un bono cero cupón a 2Y con un strike de 98.3%.
Entregue el resultado 8 decimales y suponiendo un nocional de 1.


Pregunta 2:

Ceteris paribus una opción sobre una activo con reversión a la media vale menos
que una opción sobre un activo sin reversión a la media. Comente.


Pregunta 3:

Calcule la TNA con ICP0 = 10,000.00 e ICP365 = 10,378.00 (365 días después).


Pregunta 4:

Con la curva cero cupón, interpolando linealmente en tasa, calcule la tasa
forward entre 482 y 654 días.


Pregunta 5:

Explique la diferencia entre valorizar un payoff g(r(T), T) por simulación de
Montecarlo utilizando el modelo de Hull-White en la medida libre de riesgo y el
modelo de Hull-White en la medida T-forward.


Pregunta 6:

Calcule un paso de simulación de Montecarlo para el modelo de Vasicek usando
gamma = 1.0, sigma = 0.5%, dt = 1/264, r0 obtenido de la curva cupón cero
entregada y

## Respuestas

### Pregunta 1

In [53]:
gamma=1
sigma=0.005
k=.983
CALL = hw.zcb_call_put(
        hw.CallPut.CALL,
        k,
        curva(.0001),
        1,
        2,
        math.exp(-curva(365)),
        math.exp(-curva(730) * 730 / 365.0),
        gamma,
        sigma
        )

print(f'{CALL}')

0.016555682193475696


### Pregunta 2

Un activo con reversion a la media vale memos, que uno sin reversion,  ASR > ACR
esto se debe a que la reversion a la media disminuye la volatilidad realizada, aun que $\sigma$ sea igual.

### Pregunta 3

In [31]:
dias= 365
TNA = ( (10378/10000)-1 )*(360/dias)
print(f'{TNA}')

0.03728219178082197


### Pregunta 4

In [34]:
from scipy.interpolate import interp1d
import math

crv=interp1d(df_curva["plazo"],df_curva["tasa"],"linear")

t1=482
t2=654
r1=crv(t1)
r2=crv(t2)
df1=math.exp(-r1*t1/365)
df2=math.exp(-r2*t2/365)
fdw12=(df1/df2-1.0)*360.0/(t2-t1)

print(f'{fdw12}')

0.0004105425284616809


### Pregunta 5

La principal diferencia es la forma con la que se trae a valor presente.
En la medida libre de riesgo se utiliza $\theta_t$, y se descuenta dia a dia, mientras que en la medida t_Foward simula con un $\theta_t$ diferente,
que es $\theta_t_gorro$, y para traer a valor presente con la tasa_Foward se utiliza la tasa del periodo. 


### Pregunta 6

In [40]:
from typing import List, Tuple
import scipy.optimize as opt
import plotly.express as px
import pandas as pd
import numpy as np
import math

frmt = {'tasa': '{:.4%}', 'df': '{:.6%}'}

gamma = 1
sigma = .005
r_ = .005
#r0 = .05 # tasa corta a día de hoy
#CALCULO R0 momento t=0

curva = pd.read_excel('data/20201012_built_sofr_zero.xlsx')

frmt = {'tasa': '{:.4%}', 'df': '{:.6%}'}
#curva.head().style.format(frmt)
curva['t'] = curva['plazo'] / 365.0
curva['rate'] = np.log(1 / curva['df'])/( curva['plazo'] / 365.0)
#curva.head().style.format(frmt)
zcurva = interp1d(curva['t'],
                  curva['rate'],
                  kind='cubic',
                  fill_value="extrapolate")
r0= zcurva(0)

def vasicek_path(r0: float,
                 gamma: float, r_: float,
                 sigma: float,
                 num_dias: int = 263,
                 dias_agno: int = 264,
                 pasos_dia: int = 1) -> List[Tuple[float, float]]:
    """
    Retorna un camino de simulación del modelo de Vasicek.
    
    params:
    
    - r0: tasa inicial (t = 0) de la simulación
    - Parámetros del modelo:
      - gamma: velocidad de reversión
      - r_: tasa de largo plazo
      - sigma: volatilidad
    - num_dias: número de días en la trayectoria, incluyendo el instante t = 0
    - dias_agno: número de días hábiles por año
    - pasos_dia: número de pasos de simulación en 1 día
    
    return:
    
    - Un `list` donde cada elemento es una `tuple` con los valores del tiempo y la tasa simulada.
    """
    dt = 1 / (dias_agno * pasos_dia)
    dt_gamma_r_ = dt * gamma * r_
    sigma_sqdt = sigma * math.sqrt(dt)
    result = [(0, r0),]
    gamma_dt = gamma * dt
    
    r = r0
    for i in range(1, (num_dias + 1) * pasos_dia):
        r = dt_gamma_r_ + (1 - gamma_dt) * r + sigma_sqdt * 0.4115 #np.random.normal() # Discretización de Euler
        result.append((i * dt, r))
    return result

vasicek_path(r0, gamma, r_, sigma, num_dias = 263, dias_agno = 264, pasos_dia = 1)

[(0, array(0.00078814)),
 (0.003787878787878788, 0.0022408528037132216),
 (0.007575757575757576, 0.003688062377755102),
 (0.011363636363636364, 0.005129790097349855),
 (0.015151515151515152, 0.006566056727097659),
 (0.01893939393939394, 0.007996882952944902),
 (0.022727272727272728, 0.009422289382482118),
 (0.026515151515151516, 0.010842296545240783),
 (0.030303030303030304, 0.012256924892988998),
 (0.03409090909090909, 0.013666194800026047),
 (0.03787878787878788, 0.015070126563475833),
 (0.04166666666666667, 0.016468740403579218),
 (0.045454545454545456, 0.017862056463985244),
 (0.04924242424242424, 0.01925009481204125),
 (0.05303030303030303, 0.020632875439081887),
 (0.05681818181818182, 0.02201041826071707),
 (0.06060606060606061, 0.023382743117118788),
 (0.06439393939393939, 0.02474986977330686),
 (0.06818181818181818, 0.026111817919433615),
 (0.07196969696969698, 0.027468607171067465),
 (0.07575757575757576, 0.02882025706947543),
 (0.07954545454545454, 0.030166787081904577),
 (0.

### Pregunta 7